# Курсовой проект по регрессии (Машинное обучение)

Укладников Андрей


In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
import random
random.seed(1)

In [3]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")
df_submission=pd.read_csv("submission.csv")

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
Id                     10000 non-null int64
age                    10000 non-null float64
years_of_experience    10000 non-null float64
lesson_price           10000 non-null float64
qualification          10000 non-null float64
physics                10000 non-null float64
chemistry              10000 non-null float64
biology                10000 non-null float64
english                10000 non-null float64
geography              10000 non-null float64
history                10000 non-null float64
mean_exam_points       10000 non-null float64
dtypes: float64(11), int64(1)
memory usage: 937.6 KB


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
Id                     10000 non-null int64
age                    10000 non-null float64
years_of_experience    10000 non-null float64
lesson_price           10000 non-null float64
qualification          10000 non-null float64
physics                10000 non-null float64
chemistry              10000 non-null float64
biology                10000 non-null float64
english                10000 non-null float64
geography              10000 non-null float64
history                10000 non-null float64
dtypes: float64(10), int64(1)
memory usage: 859.5 KB


In [6]:
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
Id                  10000 non-null int64
mean_exam_points    10000 non-null float64
dtypes: float64(1), int64(1)
memory usage: 156.4 KB


In [7]:
df_train.describe()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,45.878000,1.986800,1699.105000,1.719500,0.375000,0.132900,0.109600,0.053700,0.032100,0.019400,64.340800
std,2886.89568,8.043929,1.772213,524.886654,0.792264,0.484147,0.339484,0.312406,0.225436,0.176274,0.137933,13.536823
min,0.00000,23.000000,0.000000,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.000000
25%,2499.75000,40.000000,0.000000,1300.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,55.000000
50%,4999.50000,46.000000,2.000000,1500.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,63.000000
75%,7499.25000,51.000000,3.000000,2150.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,73.000000
max,9999.00000,68.000000,10.000000,3950.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000


In [8]:
TARGET_NAME = 'mean_exam_points'
FEATURE_NAMES = ['age','years_of_experience','lesson_price','qualification','physics'] 

X_train=df_train[FEATURE_NAMES].to_numpy()
X_test=df_test[FEATURE_NAMES].to_numpy()
Y_train=df_train[TARGET_NAME].to_numpy()

print(X_train.shape)
print(Y_train.shape)
print(X_train)
print(Y_train)

Y_test=df_submission.to_numpy()

(10000, 5)
(10000,)
[[4.00e+01 0.00e+00 1.40e+03 1.00e+00 1.00e+00]
 [4.80e+01 4.00e+00 2.85e+03 3.00e+00 1.00e+00]
 [3.90e+01 0.00e+00 1.20e+03 1.00e+00 0.00e+00]
 ...
 [3.40e+01 1.00e+00 1.25e+03 1.00e+00 1.00e+00]
 [3.30e+01 3.00e+00 1.10e+03 1.00e+00 0.00e+00]
 [3.50e+01 0.00e+00 1.45e+03 1.00e+00 0.00e+00]]
[63. 86. 53. ... 58. 51. 59.]


In [9]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [10]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels       
        self.prediction = self.predict()
        
    def predict(self):
         
        prediction = np.mean(self.labels)
        return prediction       

In [11]:
# Построение дерева с помощью рекурсивной функции
class Tree:
    
    def __init__(self,max_depth=50):        
        self.max_depth=max_depth        
        self.tree=None   
    
    # Расчет критерия Джини (дисперсии)
    def gini_var(self,labels):
        impurity=np.std(labels)
        return impurity
    
    # Расчет качества
    def quality(self, left_labels, right_labels, current_gini):
        # доля выбоки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])    
        return current_gini - p * self.gini_var(left_labels) - (1 - p) * self.gini_var(right_labels)

    # Разбиение датасета в узле
    def split(self, data, labels, index, t):    
        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)
        
        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]
        
        return true_data, false_data, true_labels, false_labels
    
    # Нахождение наилучшего разбиения
    def find_best_split(self,data, labels):    
        #  обозначим минимальное количество объектов в узле
        min_leaf = 5

        current_gini = self.gini_var(labels)

        best_quality = 0
        best_t = None
        best_index = None
    
        n_features = data.shape[1]
    
        for index in range(n_features):
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in data])
        
            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < min_leaf or len(false_data) < min_leaf:
                    continue
            
                current_quality = self.quality(true_labels, false_labels, current_gini)
            
                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index
    
    # Построение дерева с помощью рекурсивной функции     
    def build_tree(self, data, labels, curr_depth, max_depth):
       
        quality, t, index = self.find_best_split(data, labels)

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        # или глубина дерева больше максимальной 
        
        if quality == 0:            
            return Leaf(data, labels)
        
        if curr_depth >= max_depth:            
            return Leaf(data, labels)
        
        curr_depth+=1

        true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)

        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels, curr_depth, max_depth)
        false_branch = self.build_tree(false_data, false_labels, curr_depth, max_depth)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)
    
    
    def predict_object(self, obj, node):    
        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.predict_object(obj, node.true_branch)
        else:
            return self.predict_object(obj, node.false_branch)
        
    def predict(self, data):    
        vals = []
        for obj in data:
            prediction = self.predict_object(obj, self.tree)
            vals.append(prediction)
        return vals  
    
    def fit(self,data,labels):
        self.tree = self.build_tree(data,labels,0, self.max_depth)
        return self
    

In [12]:
def r_2(y_pred,y_true):
    numerator = ((y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
    denominator = ((y_true - np.average(y_true)) ** 2).sum(axis=0, dtype=np.float64)
    return 1 - (numerator / denominator)
    

In [13]:
treeObj=Tree(max_depth=20)
treeObj.fit(X_train,Y_train)
Y_predict= treeObj.predict(X_train)

In [14]:
print(r_2(Y_predict,Y_train))

0.8413996645036054


In [15]:
class StochasticGradientBoosting:
    def __init__(self, n_trees, max_depth, coefs, eta,subsample_size):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.coefs = coefs
        self.eta = eta
        self.subsample_size=subsample_size
        self.trees =[]
        
    def bias(self, y, z):
        return (y - z)
        
    def get_bootstrap(self, data, target,n_trees):
        n_samples = int(data.shape[0]*self.subsample_size)
        bootstrap_x = []
        bootstrap_y = []
    
        for i in range(n_trees):
            b_data = np.zeros((int(data.shape[0]*self.subsample_size),data.shape[1]))   
            b_y = np.zeros((int(target.shape[0]*self.subsample_size)))   
            for j in range(n_samples):
                sample_index = random.randint(0, n_samples-1)
                b_data[j] = data[sample_index]
                b_y[j] = target[sample_index]
                       
            bootstrap_x.append(b_data)
            bootstrap_y.append(b_y)
                
        return bootstrap_x,bootstrap_y

    def fit(self, X_train, y_train):    
        #Построим выборки для заданного количества деревьев
        X_train_arr, y_train_arr = self.get_bootstrap(X_train,y_train,n_trees)      
         
        # Деревья будем записывать в список
        self.trees = []
                       
        for i in range(self.n_trees):
            tree = Tree(max_depth=self.max_depth)
        
            # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
            # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
            if len(self.trees) == 0:
                # обучаем первое дерево на обучающей выборке                
                tree.fit(X_train_arr[i], y_train_arr[i])  
                
            else:
                # Получим ответы на текущей композиции
                target = self.predict(X_train_arr[i])                               
            
                # алгоритмы начиная со второго обучаем на сдвиг
                bias_step=self.bias(y_train_arr[i], target)
                tree.fit(X_train_arr[i],bias_step ) 
                

            self.trees.append(tree)
        
        return self
    
    def predict(self, X):
        # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
        # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании прибавляются с шагом eta
        return np.array([sum([self.eta * coef * alg.predict([x])[0] for alg, coef in zip(self.trees, self.coefs)]) for x in X])
        

In [16]:
train_data,test_data,train_labels,test_labels=train_test_split(X_train,Y_train,test_size=0.3, random_state=42)

In [17]:
n_trees=11
coefs=[1]*n_trees
max_depth=5
eta=0.59
subsample_size=0.99

In [18]:
sgb = StochasticGradientBoosting(n_trees, max_depth, coefs, eta,subsample_size)
sgb.fit(train_data,train_labels)
train_answers = sgb.predict(train_data)
test_answers = sgb.predict(test_data)


In [19]:
print(r_2(test_answers,test_labels))

0.7742386738339218


In [20]:
print(r_2(train_answers,train_labels))

0.7944580151689902


In [22]:
final_sgb = StochasticGradientBoosting(n_trees, max_depth, coefs, eta,subsample_size)
final_sgb.fit(X_train,Y_train)


In [23]:
Y_test = final_sgb.predict(X_test)


In [24]:
submissions=pd.concat([df_submission['Id'],pd.Series(Y_test)],axis=1)
submissions=submissions.rename(columns={0:'mean_exam_points'})
submissions.to_csv('AUkladnikov_predictions_1.csv',index=None)